In [2]:
import numpy as np

In [3]:
import requests

client_id = '121298'
client_secret = '003ac5e5560276a0a04412a5940d60aac9e0a6db'
auth_code = 'a8b9e1918c832ec3b21c0a5a4575ca67930cb791'

response = requests.post(
    url="https://www.strava.com/oauth/token",
    data={
        'client_id': client_id,
        'client_secret': client_secret,
        'code': auth_code,
        'grant_type': 'authorization_code'
    }
)

tokens = response.json()

access_token = tokens['access_token']


In [4]:
headers = {'Authorization': f'Bearer {access_token}'}
page = 1
activities = []

while True:
    response = requests.get(
        f"https://www.strava.com/api/v3/athlete/activities?page={page}&per_page=200",
        headers=headers
    )
    data = response.json()
    if not data:
        break
    activities.extend(data)
    page += 1

print(f"Downloaded {len(activities)} activities")


Downloaded 2294 activities


In [25]:
activity_id = activities[0]['id']

stream_response = requests.get(
        f"https://www.strava.com/api/v3/activities/{activity_id}/streams?keys=latlng&key_by_type=true",
        headers=headers
    )

if stream_response.status_code == 200:
    stream_data = stream_response.json()
    if 'latlng' in stream_data:
        coords = stream_data['latlng']['data']
coords

with open('data.json', 'w') as f:
    json.dump(coords, f)


In [23]:
activities[0]['latlng']

KeyError: 'latlng'

In [ ]:
# this one takes a long time to run - I need to find a way around it
# import time
# 
# all_coords = []
# 
# for activity in activities:
#     activity_id = activity['id']
#     stream_response = requests.get(
#         f"https://www.strava.com/api/v3/activities/{activity_id}/streams?keys=latlng&key_by_type=true",
#         headers=headers
#     )
# 
#     if stream_response.status_code == 200:
#         stream_data = stream_response.json()
#         if 'latlng' in stream_data:
#             coords = stream_data['latlng']['data']
#             all_coords.extend(coords)
#     time.sleep(1)  # Be nice to the API
# 
# print(f"Collected {len(all_coords)} GPS points")
# 

In [21]:
import sqlite3
import json

In [22]:
# === SETUP DATABASE ===
DB_FILE = 'strava_cache.db'
conn = sqlite3.connect(DB_FILE)
c = conn.cursor()

# Create activity table
c.execute('''
CREATE TABLE IF NOT EXISTS activities (
    id INTEGER PRIMARY KEY,
    data TEXT
)
''')

# Create stream table
c.execute('''
CREATE TABLE IF NOT EXISTS streams (
    activity_id INTEGER PRIMARY KEY,
    data TEXT
)
''')

conn.commit()

# === FUNCTIONS ===

def activity_cached(activity_id):
    c.execute('SELECT 1 FROM activities WHERE id = ?', (activity_id,))
    return c.fetchone() is not None

def stream_cached(activity_id):
    c.execute('SELECT 1 FROM streams WHERE activity_id = ?', (activity_id,))
    return c.fetchone() is not None

def cache_activity(activity_id, data):
    c.execute('INSERT OR REPLACE INTO activities (id, data) VALUES (?, ?)', (activity_id, json.dumps(data)))
    conn.commit()

def cache_stream(activity_id, data):
    c.execute('INSERT OR REPLACE INTO streams (activity_id, data) VALUES (?, ?)', (activity_id, json.dumps(data)))
    conn.commit()






for act in activities:
    act_id = act['id']
    if not activity_cached(act_id):
        cache_activity(act_id, act)
        print(f"Cached activity {act_id}")
    else:
        print(f"Activity {act_id} already cached")

    
print("Done.")
conn.close()


Cached activity 14437238947
Cached activity 14436779130
Cached activity 14429279127
Cached activity 14415920963
Cached activity 14416300313
Cached activity 14416291119
Cached activity 14396170079
Cached activity 14389889098
Cached activity 14389792070
Cached activity 14378986362
Cached activity 14366566976
Cached activity 14366500440
Cached activity 14355870390
Cached activity 14345632799
Cached activity 14337703737
Cached activity 14327528900
Cached activity 14316611153
Cached activity 14304204733
Cached activity 14294560433
Cached activity 14282600516
Cached activity 14272978214
Cached activity 14267329651
Cached activity 14253244380
Cached activity 14247116112
Cached activity 14235260635
Cached activity 14226775715
Cached activity 14214810050
Cached activity 14209493411
Cached activity 14208974584
Cached activity 14236250571
Cached activity 14208186802
Cached activity 14207085862
Cached activity 14208185168
Cached activity 14208180876
Cached activity 14176172315
Cached activity 1416

In [26]:
for act in activities:
    act_id = act['id']
    print(act_id)

14437238947
14436779130
14429279127
14415920963
14416300313
14416291119
14396170079
14389889098
14389792070
14378986362
14366566976
14366500440
14355870390
14345632799
14337703737
14327528900
14316611153
14304204733
14294560433
14282600516
14272978214
14267329651
14253244380
14247116112
14235260635
14226775715
14214810050
14209493411
14208974584
14236250571
14208186802
14207085862
14208185168
14208180876
14176172315
14169212920
14157246372
14146747391
14137192109
14137185034
14117430673
14111481308
14103804043
14103798564
14080593930
14079721877
14078468714
14073753374
14063332745
14053696233
14044126573
14034003121
14032795100
14030917585
14023851136
14021946378
14021455130
14016220899
14014302751
14001944162
13998179909
13991711518
13987297386
13977768825
13965986667
13954250405
13949186693
13936219011
13929398861
13917889202
13911927504
13911541031
13900980451
13893414465
13889057722
13881300581
13879500654
13871347337
13871342257
13857423664
13847908445
13836409058
13826158936
1382